# Fire Detection System with YOLO

In this fire detection system, it leverages a You Only Look Once (YOLO) deep learning model to identify the presence of fire in images. YOLO is a powerful object detection algorithm known for its speed and accuracy.

Fire detection system can play a crucial role in ensuring public safety and minimizing property damage. Some potential applications of YOLO-based fire detection model might help real-time fire alarms in buildings, which can be integrated into surveillance systems to trigger alarms upon detecting fire in real-time. Additionally, deploying the model with cameras strategically placed in forests can enable early detection of wildfires.

In [1]:
import os
HOME = os.getcwd()
print(HOME)

/home/cacc/Repositories/WildfireDetectionResearch/gatewaydetection


In [1]:
%pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.109 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7933MiB)
Setup complete ✅ (16 CPUs, 14.8 GB RAM, 259.0/467.9 GB disk)


In [2]:
from ultralytics import YOLO

from IPython.display import display, Image

In [10]:
%pip install roboflow

from roboflow import Roboflow

try:
    with open("config.txt", "r") as f:
        api_key = f.readline().strip()  # 读取第一行并移除空白字符
    if api_key:
        rf = Roboflow(api_key=api_key)
        project = rf.workspace("ayaramis").project("fire-vqbia")
        version = project.version(1)
        local_download_path = "./data"  # 替换为您希望保存数据的本地路径
        dataset = version.download("yolov8", location=local_download_path, overwrite=True) # 添加 overwrite=True
        print(f"数据集已下载到: {dataset.location}")
    else:
        print("Error: API key is empty in config.txt.")
except FileNotFoundError:
    print("Error: config.txt not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Note: you may need to restart the kernel to use updated packages.
An error occurred: {
    "error": {
        "message": "This API key does not exist (or has been revoked).",
        "status": 401,
        "type": "OAuthException",
        "hint": "You may retrieve your API key via the Roboflow Dashboard. Go to Account > Roboflow Keys to retrieve yours.",
        "key": "rf_Bo70Ve46s0TMFftFPv5FEnI3KKh1"
    }
}


## Custom Training

In [ ]:
pwd

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[3:7]:
      display(Image(filename=image_path, width=600))
      print("\n")